# Problem Set 1 (PS1): Pathological K-means example
The lecture introduced the [K-means clustering technique](https://en.wikipedia.org/wiki/K-means_clustering) as part of our unsupervised learning discussion. The [K-means approach](https://en.wikipedia.org/wiki/K-means_clustering) is straightforward and, in some ways, intuitive, but it has several limitations, such as the necessity to specify the number of clusters in advance. Additionally, there are more significant concerns, such as the inability to cluster overlapping datasets (or non-convex datasets). 

### Problem
There was the following pathological example given [in the MAT 180 Big Data course at UC Davis](https://www.math.ucdavis.edu/~strohmer/courses/180BigData/180coursematerial.html) that was not explored in detail:
* __Example__: Consider two circles, each of radius 1, and their centers are a distance $d$ apart.
As long as $d > 2.08$, K-means yields the correct answer. But if $d\leq{2.08}$, K-means fails.
This is the failure of K-means and not of Lloyd's algorithm.

This is what we'll explore in PS1.

## Setup, Prerequisites and Data
We set up the computational environment by including the `Include.jl` file and then load any needed resources, e.g., a sample dataset, to cluster, and we end by setting up any required constants. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

In [21]:
include("Include.jl");

[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a] (cache misses: wrong dep version loaded (20))


generatedata (generic function with 1 method)

### Data
Fill me in